In [ ]:
from pynq import Overlay
from pynq import Xlnk
import pprint
import matplotlib.pyplot as plt
import numpy as np
import cv2
from time import time
xlnk = Xlnk()
overlay = Overlay('./RS_BRIEF_ultra96-v2.bit')
dma_src = overlay.axi_dma_src
dma_des = overlay.axi_dma_des
dma_des.recvchannel.start()
dma_src.sendchannel.start()


In [ ]:
img_ori = cv2.imread('./test_data/'+'000000'+'.png')
img_gray = cv2.cvtColor(img_ori,cv2.COLOR_BGR2GRAY)
src_buf = xlnk.cma_array(shape=(1241*376,), dtype=np.uint16)
view=np.frombuffer(src_buf,dtype = np.uint16,count = -1)
np.copyto(view,img_gray.ravel(),casting='same_kind')
des_buf = xlnk.cma_array(shape=(8192,8), dtype=np.uint64)
for i in range (1241*20,1241*24):
    src_buf[i] = src_buf[i] + 0b111100000000


In [ ]:
t0 = time()
dma_des.recvchannel.transfer(des_buf)
dma_src.sendchannel.transfer(src_buf)
t1 = time()
tmp = 0
for i in range(50000):
    tmp = tmp + 1
t2 = time()
dma_src.sendchannel.wait()
dma_des.recvchannel.wait()
des_buf.flush()
t3 = time()
print('time of transfer:'+ str(t1-t0) + 'seconds')
print('time of addition:'+ str(t2-t0) + 'seconds')
print('time of computing RS BRIEF:'+ str(t3-t0) + 'seconds')
bytes_read=dma_des.mmio.read(0x58)
featurePointsNum = int(bytes_read/64) - 1
print(str(featurePointsNum)+' feature points are detected')

In [ ]:
for i in range(int(bytes_read/64)):
    for j in range(8):
        print(bin(des_buf[i][j]))

In [ ]:
dma_des.recvchannel.stop()
dma_src.sendchannel.stop()
src_buf.freebuffer()
des_buf.freebuffer()